# Latrodectus
> Extracting new AES encrypted strings from this RAT
- toc: true 
- badges: true
- categories: [Latrodectus,config,emulation]

## Overview
Latrodectus is a RAT that has been [linked to the development of IcedID](https://medium.com/walmartglobaltech/icedid-gets-loaded-af073b7b6d39). It provides host information collection capabilities as well as the ability to download and launch additional payloads.

### Sample
`5cecb26a3f33c24b92a0c8f6f5175da0664b21d7c4216a41694e4a4cad233ca8` [UnpacMe](https://www.unpac.me/results/148ce170-808d-4b22-bdaf-6470c472d782)

### References
- [Malpedia](https://malpedia.caad.fkie.fraunhofer.de/details/win.latrodectus)
- [VMRay X - New #Latrodectus version (1.4)](https://x.com/vmray/status/1823762654156018020)
- [Blu3Eye Latrodectus String Extractor (IDA)](https://github.com/Blu3Eye/Malware-Analysis/blob/master/Latrodectus/extract_decrypt_strings.py)
- [Latrodectus Rapid Evolution Continues With Latest New Payload Features](https://www.netskope.com/blog/latrodectus-rapid-evolution-continues-with-latest-new-payload-features)
- [Icicle version of pemulator and decrytion](https://gist.github.com/mrexodia/632561f6e96706be2a40ffab23282c28)

## Analysis

The focus of our analysis will be to extract the new AES encrypted strings. 

We are going to use the [pemulator.py](https://gist.github.com/herrcore/d371e1b8cd5730d57e5a4dc4cba4d962) lib to assist with emulation.


In [1]:
from pemulator import Emulator

In [3]:
file_path = '/tmp/5cecb26a3f33c24b92a0c8f6f5175da0664b21d7c4216a41694e4a4cad233ca8'
file_data = open(file_path, 'rb').read()

In [4]:
from unicorn import *
from unicorn.x86_const import *
import struct 

emu = Emulator(file_data, mode=64, trace=False)

scratch_start = emu.get_memory_upper_bound() + 0x1000
emu.emu.mem_map(scratch_start, 0x1000)

buff = scratch_start

image_base = emu._pe.OPTIONAL_HEADER.ImageBase
data_addr = 0x01400110A8
dec_fn = 0x014000C1E4

# Move buff into rdx
emu.emu.reg_write(UC_X86_REG_RDX, buff)

# Move enc string into 
emu.emu.reg_write(UC_X86_REG_RCX, data_addr)

# address to end emulation
kill_addr = 0xdeadbeef
kill_addr_data = struct.pack('<I', kill_addr)

esp = emu.emu.reg_read(UC_X86_REG_RSP)
emu.emu.mem_write(esp, kill_addr_data)

emu.emu.emu_start(dec_fn, kill_addr)

tmp_string = emu.emu.mem_read(buff, 0x1000).replace(b'\x00',b'')
print(tmp_string)


bytearray(b'POST')


In [5]:
def decrypt(data_addr):
    emu = Emulator(file_data, mode=64, trace=False)
    
    scratch_start = emu.get_memory_upper_bound() + 0x1000
    emu.emu.mem_map(scratch_start, 0x1000)
    
    buff = scratch_start
    
    image_base = emu._pe.OPTIONAL_HEADER.ImageBase
    #data_addr = 0x01400110A8
    dec_fn = 0x014000C1E4
    
    # Move buff into rdx
    emu.emu.reg_write(UC_X86_REG_RDX, buff)
    
    # Move enc string into 
    emu.emu.reg_write(UC_X86_REG_RCX, data_addr)
    
    # address to end emulation
    kill_addr = 0xdeadbeef
    kill_addr_data = struct.pack('<I', kill_addr)
    
    esp = emu.emu.reg_read(UC_X86_REG_RSP)
    emu.emu.mem_write(esp, kill_addr_data)
    
    emu.emu.emu_start(dec_fn, kill_addr)
    
    tmp_string = emu.emu.mem_read(buff, 0x1000).replace(b'\x00',b'')
    return tmp_string

decrypt(0x01400110A8)

bytearray(b'POST')

In [12]:
import re

tmp_emu = Emulator(file_data, mode=64, trace=False)

# Find all refs to the enc strings
#
# .text:0000000140005E19 48 8D 94 24 80 00 00 00                 lea     rdx, [rsp+188h+var_108]
# .text:0000000140005E21 48 8D 0D 80 B2 00 00                    lea     rcx, word_1400110A8
# .text:0000000140005E28 E8 B7 63 00 00                          call    latro_str_decryption_main
#
# .text:0000000140007A83 48 8D 54 24 40                          lea     rdx, [rsp+88h+var_48]
# .text:0000000140007A88 48 8D 0D 91 A1 00 00                    lea     rcx, word_140011C20
# .text:0000000140007A8F E8 50 47 00 00                          call    latro_str_decryption_main

egg = rb'\x48\x8D.{3,6}\x48\x8D\x0D(....)\xE8'

for match in re.finditer(egg, file_data):
    match_len = len(match.group())
    if match_len == 13:
        # Smaller offset
        image_base = tmp_emu._pe.OPTIONAL_HEADER.ImageBase
        start_address = tmp_emu._pe.get_rva_from_offset(match.start()) + image_base
        data_addr = struct.unpack('<I', match.group(1))[0] + 12 + start_address
    else:
        image_base = tmp_emu._pe.OPTIONAL_HEADER.ImageBase
        start_address = tmp_emu._pe.get_rva_from_offset(match.start()) + image_base
        data_addr = struct.unpack('<I', match.group(1))[0] + 15 + start_address
    #print(f"{hex(start_address)}: STRING: {hex(data_addr)}")
    try:
        tmp_string = decrypt(data_addr)
        if tmp_string.isascii():
            out = tmp_string.decode('utf-8')
            print(f"{hex(data_addr)}: {out}")
        else:
            print(f"{hex(data_addr)}: -------> {tmp_string}")
    except:
        print(f"{hex(data_addr)}: ***ERROR - no string")
        




0x1400109f8: {
0x140010a10: "pid": 
0x140010a30: "%d",
0x140010a50: "proc": 
0x140010a70: "%s",
0x140010a90: "subproc": [
0x140010ab8: ]
0x140010ad0: }
0x140010c00: &desklinks=[
0x140010c28: *.*
0x140010c48: "%s"
0x140010c68: ]
0x140010ae8: &proclist=[
0x140010b10: {
0x140010b28: "pid": 
0x140010b48: "%d",
0x140010b68: "proc": 
0x140010b88: "%s",
0x140010ba8: "subproc": [
0x140010bd0: ]
0x140010be8: }
0x140010000: /c ipconfig /all
0x140010070: C:\Windows\System32\cmd.exe
0x140010038: /c systeminfo
0x1400100c0: C:\Windows\System32\cmd.exe
0x140010110: /c nltest /domain_trusts
0x140010190: C:\Windows\System32\cmd.exe
0x1400101e0: /c nltest /domain_trusts /all_trusts
0x140010240: C:\Windows\System32\cmd.exe
0x140010290: /c net view /all /domain
0x140010300: C:\Windows\System32\cmd.exe
0x140010158: /c net view /all
0x140010350: C:\Windows\System32\cmd.exe
0x1400103a0: /c net group "Domain Admins" /domain
0x140010400: C:\Windows\System32\cmd.exe
0x140010450: /Node:localhost /Namespace:\\roo

In [37]:
hex(0x0140007A8F + 0xa191)

'0x140011c20'

'0x140000000'